In [29]:
from keras.optimizers import SGD, Adam
from keras.applications import *
from keras.callbacks import *

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.models import Model
from keras import optimizers
import numpy as np
import shap
import keras.backend as K
import json
import split_folders
import urllib
import os

In [2]:
split_folders.ratio('data/', output="output", seed=1337, ratio=(.8, .2))

In [3]:
urllib.request.urlretrieve("https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels.h5", "resnet50_weights_tf.h5")                     



('resnet50_weights_tf.h5', <http.client.HTTPMessage at 0x7f412c199860>)

In [4]:
target_size = 244

In [ ]:
# load pre-trained model and choose two images to explain
model = VGG16(weights='imagenet', include_top=True) ## CHANGE INCLUDE TOP TO FALSE
X,y = shap.datasets.imagenet50()
to_explain = X[[6,7]]

# load the ImageNet class names
url = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
fname = shap.datasets.cache(url)
with open(fname) as f:
    class_names = json.load(f)

# explain how the input to the 7th layer of the model explains the top two classes
def map2layer(x, layer):
    feed_dict = dict(zip([model.layers[0].input], [preprocess_input(x.copy())]))
    return K.get_session().run(model.layers[layer].input, feed_dict)
e = shap.GradientExplainer((model.layers[7].input, model.layers[-1].output), map2layer(preprocess_input(X.copy()), 7))
shap_values,indexes = e.shap_values(map2layer(to_explain, 7), ranked_outputs=2)

# get the names for the classes
index_names = np.vectorize(lambda x: class_names[str(x)][1])(indexes)

# plot the explanations
shap.image_plot(shap_values, to_explain, index_names)

In [ ]:
shap.image_plot(shap_values, to_explain, index_names)

In [5]:
img_width = target_size
img_height = target_size
batch_size =16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling (we do not want to modify the testing data)
test_datagen = ImageDataGenerator(rescale=1. / 255)


# The generator object. 
train_generator = train_datagen.flow_from_directory(
    'output/train',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    'output/val',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 3065 images belonging to 14 classes.
Found 774 images belonging to 14 classes.


In [12]:
base_model = resnet50.ResNet50(include_top=False, weights='imagenet',
                       input_tensor=None, input_shape=(target_size,target_size,3), pooling=None, classes=14)

TRAIN_DIR = "output/train"
HEIGHT = target_size
WIDTH = target_size
BATCH_SIZE = 8

train_datagen =  ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=90,
      horizontal_flip=True,
      vertical_flip=True
    )

train_generator = train_datagen.flow_from_directory(TRAIN_DIR, 
                                                    target_size=(HEIGHT, WIDTH), 
                                                    batch_size=BATCH_SIZE)

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Found 3065 images belonging to 14 classes.


In [13]:
def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

class_list = train_generator.class_indices
FC_LAYERS = [1024, 1024]
dropout = 0.5

finetune_model = build_finetune_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=len(class_list))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [31]:
if not os.path.exists('checkpoints'):
        os.mkdir('checkpoints')

In [35]:
NUM_EPOCHS = 5000
BATCH_SIZE = 8
num_train_images = 3065

adam = Adam(lr=0.0001) # was 0.00001
finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

filepath="./checkpoints/" + "ResNet50" + "_model_weights.h5"
checkpoint = ModelCheckpoint(filepath, monitor=["acc"], verbose=1, mode='max')
callbacks_list = [checkpoint]

history = finetune_model.fit_generator(train_generator, epochs=NUM_EPOCHS, workers=8, 
                                       steps_per_epoch=num_train_images // BATCH_SIZE, 
                                       shuffle=True, callbacks=callbacks_list)

Epoch 1/5000
 31/383 [=>............................] - ETA: 1:42 - loss: 14.4818 - acc: 0.1008

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/PIL/Image.py:931: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


214/383 [===============>..............] - ETA: 14s - loss: 14.7571 - acc: 0.0841

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 6 bytes but only got 5. Skipping tag 42036
  " Skipping tag %s" % (size, len(data), tag))


383/383 [==============================] - 27s 71ms/step - loss: 14.9053 - acc: 0.0751

Epoch 00001: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 2/5000
383/383 [==============================] - 21s 54ms/step - loss: 14.8661 - acc: 0.0777

Epoch 00002: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 3/5000
383/383 [==============================] - 20s 52ms/step - loss: 14.8714 - acc: 0.0774

Epoch 00003: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 4/5000
383/383 [==============================] - 21s 54ms/step - loss: 14.9029 - acc: 0.0754

Epoch 00004: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 5/5000
383/383 [==============================] - 21s 54ms/step - loss: 14.8556 - acc: 0.0783

Epoch 00005: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 6/5000
383/383 [==============================] - 20s 52ms/step - loss: 14.8819 - acc: 0.0767

Epoch 00006: saving model to ./checkpoints/ResNet50_model_weig

KeyboardInterrupt: 

In [24]:
nb_epoch = 1200

finetune_model.fit_generator(
    train_generator,
    epochs=nb_epoch,
    steps_per_epoch=100,
    validation_data=validation_generator,
    validation_steps=10)

Epoch 1/1200
  1/100 [..............................] - ETA: 18s - loss: 14.1033 - acc: 0.1250

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/PIL/Image.py:931: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


100/100 [==============================] - 23s 234ms/step - loss: 14.7681 - acc: 0.0838 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 2/1200
100/100 [==============================] - 23s 228ms/step - loss: 15.0503 - acc: 0.0663 - val_loss: 14.7212 - val_acc: 0.0867
Epoch 3/1200
100/100 [==============================] - 22s 221ms/step - loss: 14.7481 - acc: 0.0850 - val_loss: 15.0100 - val_acc: 0.0688
Epoch 4/1200
 12/100 [==>...........................] - ETA: 17s - loss: 14.6070 - acc: 0.0938

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 6 bytes but only got 5. Skipping tag 42036
  " Skipping tag %s" % (size, len(data), tag))


100/100 [==============================] - 21s 213ms/step - loss: 14.8488 - acc: 0.0788 - val_loss: 14.9092 - val_acc: 0.0750
Epoch 5/1200
100/100 [==============================] - 21s 212ms/step - loss: 15.0502 - acc: 0.0663 - val_loss: 15.3122 - val_acc: 0.0500
Epoch 6/1200
100/100 [==============================] - 23s 233ms/step - loss: 14.7884 - acc: 0.0825 - val_loss: 14.7078 - val_acc: 0.0875
Epoch 7/1200
100/100 [==============================] - 23s 233ms/step - loss: 14.8488 - acc: 0.0788 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 8/1200
100/100 [==============================] - 21s 206ms/step - loss: 14.9495 - acc: 0.0725 - val_loss: 15.1107 - val_acc: 0.0625
Epoch 9/1200
100/100 [==============================] - 23s 232ms/step - loss: 14.7882 - acc: 0.0825 - val_loss: 14.9092 - val_acc: 0.0750
Epoch 10/1200
100/100 [==============================] - 23s 228ms/step - loss: 14.9092 - acc: 0.0750 - val_loss: 14.7078 - val_acc: 0.0875
Epoch 11/1200
100/100 [================

KeyboardInterrupt: 

In [19]:
img = load_img('output/val/boxer/1485179194-boxer-dog-portrait.jpg',target_size=(target_size,target_size) )
img = img_to_array(img)
img = img.reshape((1,) + img.shape)
img.shape

(1, 244, 244, 3)

In [21]:
pred = finetune_model.predict(img)
pred

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)

In [22]:
np.round(pred, decimals=3)

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)

In [23]:
predicted_class_indices=np.argmax(pred,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
predictions

['beagle']

In [ ]:
# load pre-trained model and choose two images to explain
# model = VGG16(weights='imagenet', include_top=True) ## CHANGE INCLUDE TOP TO FALSE
model = tran_model
# X,y = shap.datasets.imagenet50()
img = load_img('output/val/boxer/1485179194-boxer-dog-portrait.jpg',target_size=(224,224) )
img = img_to_array(img)
img = img.reshape((1,) + img.shape)
to_explain = img
X = img

# # load the ImageNet class names
# url = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
# fname = shap.datasets.cache(url)
# with open(fname) as f:
#     class_names = json.load(f)
class_names = labels

# explain how the input to the 7th layer of the model explains the top two classes
def map2layer(x, layer):
    feed_dict = dict(zip([model.layers[0].input], [preprocess_input(x.copy())]))
    return K.get_session().run(model.layers[layer].input, feed_dict)

e = shap.GradientExplainer((model.layers[7].input, model.layers[-1].output), map2layer(preprocess_input(X.copy()), 7))
shap_values,indexes = e.shap_values(map2layer(to_explain, 7), ranked_outputs=2)

# get the names for the classes
index_names = np.vectorize(lambda x: class_names[str(x)][1])(indexes)

# plot the explanations
shap.image_plot(shap_values, to_explain, index_names)

In [ ]:
K.get_session().run(model.layers[layer].input, feed_dict)